## Notebook to convert SMP models to onnx and verify model results


In [ ]:
import os
import segmentation_models_pytorch as smp
from catalyst.utils import unpack_checkpoint, load_checkpoint
import torch 
import cv2
from omegaconf import DictConfig, OmegaConf
import matplotlib.pyplot as plt
import math


## Create model architecture

In [ ]:
log_dir = "../outputs/2022-01-17/03-50-40/"

cfg = OmegaConf.load(os.path.join(log_dir, ".hydra/config.yaml"))


#load model
model = eval(
    f"smp.{cfg.model.ARCH}(encoder_name='{cfg.model.ENCODER}',encoder_weights='{cfg.model.ENCODER_WEIGHTS}',classes={len(cfg.model.CLASSES)},activation=('{cfg.model.ACTIVATION}'))"
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(cfg.model.ENCODER, cfg.model.ENCODER_WEIGHTS)
preprocessing_fn

## Load checkpoint

In [ ]:
#load checkpoint
checkpoint = load_checkpoint(
    path=os.path.join(log_dir, cfg.training.LOG_DIR, "best_full.pth")
)

#remove unwanted value pairs
unpack_checkpoint(
    checkpoint=checkpoint,
    model=model,
)

## Export to onnx

In [ ]:
model_output_dir = "../deployment/models/"
os.makedirs(model_output_dir,exist_ok = True)

In [ ]:
dummy_input = torch.randn(1, 3, 320, 320, device='cpu')
torch.onnx.export(model, dummy_input, os.path.join(model_output_dir,"model.onnx"), input_names=["input"], output_names=["output"], verbose=False,opset_version=11)

## Test onnx model

In [ ]:
import onnxruntime as rt
import numpy as np

model = rt.InferenceSession(os.path.join(model_output_dir,"model.onnx"))
input_name = model.get_inputs()[0].name
label_name = model.get_outputs()[0].name
print(label_name,input_name)

In [ ]:
image = cv2.imread("../data/training_data/train/midaxial1775.png")

# 255107942.0_TXZJEF00000806NeighOrtho00000794N_190208.jpg
#./data/benchmark_data/images/258173907.0_TXZSAN00000906NeighOrtho00002531N_190128.jpg
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
org_img = image.copy()

### Preprocess image

In [ ]:
def preprocess(image,image_size = (320,320)):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]

    image = cv2.resize(image,image_size)
    # image_viz =image.copy()
    image = image.astype('float32')/255
    image -=mean
    image /=std
    image = image.transpose(2, 0, 1)

    image = np.expand_dims(image, axis=0)
    # image.shape
    return image

image= preprocess(image)

### Infer from onnx model

In [ ]:
pred_mask = model.run(None, {'input': image})[0]
pred_mask.shape

### Post process mask